In [1]:
# Standard imports
import cv2
import numpy as np;
import matplotlib.pyplot as plt
import PIL.Image as Image
from PIL import Image, ImageOps

In [2]:
# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()
 
# Change thresholds
params.minThreshold = 10;
params.maxThreshold = 200;
 
# Filter by Area.
params.filterByArea = True
params.minArea = 400
 
# Filter by Circularity
params.filterByCircularity = True
params.minCircularity = 0.3
 
# Filter by Convexity
params.filterByConvexity = False
params.minConvexity = 0.1
 
# Filter by Inertia
params.filterByInertia = False

params.minInertiaRatio = 0.05
 
detector = cv2.SimpleBlobDetector_create(params)

In [6]:
import cv2
import os
import numpy as np

# define a video capture object
cam = cv2.VideoCapture(1, cv2.CAP_DSHOW)



try:
    if not os.path.exists('Shranjeni frame-i'): #pogleda če obstaja ta mapa
        os.makedirs('Shranjeni frame-i') #naredi novo mapo
  
except OSError:
    print ('Error: Creating directory of data') #error, če ni možno naarediti mape 



def slikeIzVidea(st_slik, frekvenca):

    currentframe = 0
    shranjene_slike = 0

    while (True):

        ret, frame = cam.read()
        cv2.imshow('frame', frame) #prikaži trenutni frame (prikaz kamere)
        if cv2.waitKey(1) & 0xFF == ord('q'): #gumb q ustavi
            break
        if ret:
            if not (currentframe % frekvenca):
                name = './Shranjeni frame-i/frame' + str(currentframe) + '.jpg'
                print ('Creating...' + name)
                #matrika = np.array(frame)
                #matrika[:,:,2] = np.zeros([matrika.shape[0], matrika.shape[1]]) #izbriše vse rdeče kanale
                #rdeca = Image.fromarray(matrika)
                cv2.imwrite(name, frame)
                shranjene_slike += 1
                if shranjene_slike == st_slik:
                    break

            currentframe += 1
        else:
            break
        
    cam.release()
    cv2.destroyAllWindows()

In [7]:
slikeIzVidea(2,1) #(ŠTEVILO SLIK, KOLIKO FRAMOV NARAZEN (FREKVENCA))

Creating..../Shranjeni frame-i/frame0.jpg
Creating..../Shranjeni frame-i/frame1.jpg


In [8]:
# Read image
im = cv2.imread("Shranjeni frame-i/frame0.jpg")
#cv2.imshow("im",cv2.imread("Shranjeni frame-i/frame0.jpg"))

alpha = 3 # Contrast control (1.0-3.0) #3 najde kr vredu
beta = 25 # Brightness control (0-100) #25 nqjde kr vredu

adjusted = cv2.convertScaleAbs(im, alpha=alpha, beta=beta)
im = adjusted
cv2.imshow("im",im)

cv2.waitKey(0)
im1=im
hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
lower_red = np.array([0,75,200])
upper_red = np.array([255,255,255])
    
mask = cv2.inRange(hsv, lower_red, upper_red)

x_slika = []
y_slika = []

#cv2.imshow('mask',mask)

#print(mask)
#cv2.waitKey(0)

In [4]:
#im = cv2.imread("./blob.jpg", cv2.IMREAD_GRAYSCALE)
im = cv2.bitwise_not(mask)
#im=mask
# Set up the detector with default parameters.
detector = cv2.SimpleBlobDetector_create(params)
#rgb = cv2.cvtColor(im, cv2.COLOR_HSV2RGB)
#gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
 
# Detect blobs.
keypoints = detector.detect(im)
 
# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#print(keypoints)
# Show keypoints
stevnik = 0
for keyPoint in keypoints:
    x = keyPoint.pt[0]
    y = keyPoint.pt[1]
    s = keyPoint.size
    if stevnik == 0:
        x_ = keyPoint.pt[0]
        y_ = keyPoint.pt[1]
        stevnik = 1
        
    x_slika.append(x)
    y_slika.append(y)
        
    print(x)
    print(y)
    #print(s)


cv2.imshow("Keypoints", im_with_keypoints)
cv2.waitKey(0)

335.18719482421875
236.12344360351562
296.4700927734375
236.61827087402344


-1

Pretvorim v pixle na robotu

In [5]:
x_ref = 432
y_ref = 330

x_rob = []
y_rob = []

for keyPoint in keypoints:
    x_vmesni = x_ref - keyPoint.pt[0]
    y_vmesni = y_ref - keyPoint.pt[1]
    x_rob.append(y_vmesni)
    y_rob.append(x_vmesni)





In [6]:
img_camera = np.ones_like(im)
img_camera = img_camera*255


x_rob_pixli = x_rob.copy()
y_rob_pixli = y_rob.copy()

skalirni_faktor = 1.57

x_rob_mm = []
y_rob_mm = []
for i in range(len(x_rob_pixli)):
    x = x_rob_pixli[i]*skalirni_faktor
    y = y_rob_pixli[i]*skalirni_faktor
    x_rob_mm.append(x)
    y_rob_mm.append(y)

print(x_rob_mm)
print(y_rob_mm)

[147.38619354248047, 146.60931472778321]
[151.99610412597656, 212.78195434570313]


cmd /k "curl "192.167.1.20/KAREL/setprc?str_coord1=10.0&str_coord2=20.0&str_coord3=0.0&str_coord4=0.0&str_coord5=0.0&str_coord6=0.0""

In [2]:
import os
x=30.5
y=15.5
def write_to_cmd(x, y):
    set_str = f'cmd /k curl "192.167.1.20/KAREL/setprc?str_coord1={x}&str_coord2={y}&str_coord3=0.0&str_coord4=0.0&str_coord5=0.0&str_coord6=0.0".format(x,y)'
    os.system(set_str.format(x,y))
    print(set_str)
    return

In [19]:
write_to_cmd(x_rob_mm[0], y_rob_mm[0])